In [1]:
from ipynb.fs.full.common import *

Variables

In [2]:
ACTIVE_SHAM = 'Active'
SAMPLE = 'Preeti'
RANGE = 2
mapping = {0: 'delta', 1: 'theta', 2: 'alpha1', 3:'alpha2', 4:'beta1', 5:'beta2', 6: 'beta3', 7: 'gamma'}


Importing all packages

In [3]:
import numpy as np
import pandas as pd
import mne
import matplotlib
import matplotlib.pyplot as plt
import os
from scipy.stats import ttest_ind
from mne_connectivity import spectral_connectivity_epochs
import networkx as nx
import numpy as np 

matplotlib.use('TkAgg')

## LOAD DATA

In [4]:
# 2Pre-TDCS : .easy, # 3TDCS : .easy, # 4Post-TDCS : .easy
import glob
folder_path = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', '32electrodes', ACTIVE_SHAM)
# folders = ['1Pre-BML', '2Pre-TDCS', '3TDCS', '4Post-TDCS', '5Post-BML']
folders = ['2Pre-TDCS', '3TDCS', '4Post-TDCS']
all_files = []
for folder in folders:
    ext = '.eeg' if folder[0] in ['1', '5'] else 'Close.easy'
    is_even = 1
    for _ in glob.glob(os.path.join(folder_path, folder, SAMPLE) + '/*' + ext):
        if (folder[0] == '3' and is_even%2==1) or folder[0] in ['2', '4']: #TODO: add for both 1 and 5 also 
            all_files.append(_)
        is_even += 1

    sorted_files = sorted(all_files, key=lambda x: os.path.getmtime(os.path.join(folder_path, folder, SAMPLE, x)))
print(len(sorted_files), sorted_files)
# TODO: ADD iterator that changes RANGE value and iterates (len(sorted_files) - 1 times)
GROUP1 = RANGE
raw_1 = data_transformation_easy(sorted_files[RANGE])
raw_1 = g1_preprocess(raw_1)
GROUP2 = RANGE+1
raw_2 = data_transformation_easy(sorted_files[RANGE+1])
raw_2 = g1_preprocess(raw_2)

25 ['/home/vishwani/Downloads/IITD/Depression-IITD/Depression-Sample-dataset-AIIMS/32electrodes/Active/2Pre-TDCS/Preeti/20230718201550_Preeti singh_22.08.23-01_Eye Close.easy', '/home/vishwani/Downloads/IITD/Depression-IITD/Depression-Sample-dataset-AIIMS/32electrodes/Active/3TDCS/Preeti/20230718203406_Preeti singh_22.08.23_01_Rest_Eye Close.easy', '/home/vishwani/Downloads/IITD/Depression-IITD/Depression-Sample-dataset-AIIMS/32electrodes/Active/3TDCS/Preeti/20230718210007_Preeti singh_22.08.23_01_Rest_Eye Close.easy', '/home/vishwani/Downloads/IITD/Depression-IITD/Depression-Sample-dataset-AIIMS/32electrodes/Active/3TDCS/Preeti/20230719001503_PreetiSingh_22.08.23_20_02_Rest_Eye Close.easy', '/home/vishwani/Downloads/IITD/Depression-IITD/Depression-Sample-dataset-AIIMS/32electrodes/Active/3TDCS/Preeti/20230719024319_PreetiSingh_23.08.23_03_Rest_Eye Close.easy', '/home/vishwani/Downloads/IITD/Depression-IITD/Depression-Sample-dataset-AIIMS/32electrodes/Active/3TDCS/Preeti/20230719030920

/home/vishwani/Downloads/IITD/Depression-IITD/common.ipynb:253: RuntimeWarning: filter_length (155001) is longer than the signal (50001), distortion is likely. Reduce filter length or filter a longer signal.
  ]
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Creating RawArray with float64 data, n_channels=32, n_times=149999
    Range : 0 ... 149998 =      0.000 ...   299.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.01 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hann window with 0.0546 passband ripple and 44 dB stopband attenuation
- Lower transition bandwidth: 0.01 Hz
- Upper transition bandwidth: 11.25 Hz
- Filter length: 155001 samples (310.002 s)



/home/vishwani/Downloads/IITD/Depression-IITD/common.ipynb:253: RuntimeWarning: filter_length (155001) is longer than the signal (50001), distortion is likely. Reduce filter length or filter a longer signal.
  ]


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Excel data

In [5]:
existing_file = "comparison-results/results.xlsx"
df = pd.read_excel(existing_file, header=[0,1])

# main_column_names = ['index', 'swi', 'links', 'asymmetry', 'regions']
# subcols = ('delta-g1', 'delta-g2', 'theta-g1', 'theta-g2', 'alpha1-g1', 'alpha1-g2', 'alpha2-g1', 'alpha2-g2', 
#            'beta1-g1', 'beta1-g2', 'beta2-g1', 'beta2-g2', 'beta3-g1', 'beta3-g2', 'gamma-g1', 'gamma-g2')

# subcolumn_names = {
#     'index' : ['index'],
#     'swi': subcols,
#     'links': subcols,
#     'asymmetry': subcols,
#     'regions': ('delta', 'theta', 'alpha1', 'alpha2', 'beta1', 'beta2', 'beta3', 'gamma')
# }

# columns_tuples = [(main_col, sub_col) for main_col in main_column_names for sub_col in subcolumn_names[main_col]]
# columns = pd.MultiIndex.from_tuples(columns_tuples)
# df = pd.DataFrame(columns=columns)

row_data = {('index', 'index'): f'{ACTIVE_SHAM}-{SAMPLE}-{RANGE}vs{RANGE+1}'}
(df)

Unnamed: 0_level_0               index       swi                     \
  Unnamed: 0_level_1               index  delta-g1 delta-g2  theta-g1   
0                NaN                 NaN       NaN      NaN       NaN   
1                0.0  Active-Preeti-1vs2  0.191838  0.03682  0.228776   

                                                     ... asymmetry           \
   theta-g2 alpha1-g1 alpha1-g2 alpha2-g1 alpha2-g2  ...  gamma-g1 gamma-g2   
0       NaN       NaN       NaN       NaN       NaN  ...       NaN      NaN   
1  0.034939       NaN       NaN       NaN       NaN  ...  1.037037   1.3125   

  regions                                                  
    delta theta   alpha1 alpha2   beta1 beta2 beta3 gamma  
0     NaN   NaN      NaN    NaN     NaN   NaN   NaN   NaN  
1      []    []  ['RPO']     []  ['RC']    []    []    []  

[2 rows x 58 columns]

### Epoching

In [6]:
duration = 4.0
overlap = 2.0 

samples_per_epoch = int(duration * SFREQ)
samples_per_overlap = int(overlap * SFREQ)

# Manually created events
start, stop = 0, samples_per_epoch
events = []
while stop <= len(raw_1):
    events.append([start, 0, 1]) 
    start += samples_per_overlap
    stop += samples_per_overlap

events = np.array(events)

3.1 Segmenting epochs for Group 1

In [7]:
epochs_1 = mne.Epochs(raw_1, events, tmin=0, tmax=duration, baseline=None, detrend=1,
                    picks=None, preload=True, reject=None)
# raw_1.plot(n_channels=len(raw_1.info['ch_names']), events=events, event_color={1:'r'}, scalings=SCALINGS)
# epochs_1.plot(n_channels=len(raw_1.info['ch_names']), event_color={1:'r'}, events=events, scalings=SCALINGS)

Not setting metadata
49 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
15 bad epochs dropped


3.2 Segmenting epochs for Group 2

In [8]:
epochs_2 = mne.Epochs(raw_2, events, tmin=0, tmax=duration, baseline=None, detrend=1,
                    picks=None, preload=True, reject=None)
# raw_2.plot(n_channels=len(raw_1.info['ch_names']), events=events, event_color={1:'r'}, scalings=SCALINGS)
# epochs_2.plot(n_channels=len(raw_1.info['ch_names']), event_color={1:'r'}, events=events, scalings=SCALINGS)

Not setting metadata
49 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
15 bad epochs dropped


### 1. PSD graph plot between MDD and Control group

For resting state EEG channel, analyzing the overall average power across channels provides a holistic view of the brain's activity without emphasizing the specificity of individual channels.

In [9]:
# Amplitude plots between both groups
epochs_1_avg = epochs_1['1'].average()
epochs_2_avg = epochs_2['1'].average()
# evokeds = dict(epochs_1=epochs_1)
# fig_MDD = epochs_1_avg.plot(titles=GROUP1, time_unit = 'ms')
# fig_control = epochs_2_avg.plot(titles=GROUP2, time_unit = 'ms')

1.1 PSD using plot_psd

In [10]:
fig, ax = plt.subplots()
#plot_psd uses welch method for continuous data
raw_1.plot_psd(picks=raw_1.info['ch_names'], average=True, fmin=FMIN, fmax=FMAX, ax=ax, show=False, color='blue', dB=False)
raw_2.plot_psd(picks=raw_2.info['ch_names'], average=True, fmin=FMIN, fmax=FMAX, ax=ax, show=False, color='red', dB=False)

# dB = True plots PSD in decibels (logarithmic)
# different n_fft compared to psd_array_welch
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Power/Frequency (microV^2/Hz)')
ax.set_title('Power Spectral Density Comparison')

ax.text(0.8, 0.9, GROUP1, color='blue', transform=ax.transAxes)
ax.text(0.8, 0.85, GROUP2, color='red', transform=ax.transAxes)

plt.ylim(0, 8)
# plt.show()

fig.savefig(f'comparison-results/PSD - {GROUP1} vs {GROUP2}.png')

NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 4.096 (s)
NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 4.096 (s)


If p value < 0.005, we reject the null hypothesis

### 2. PLI and construction of brain function matrices

In [11]:
# pli method always gives positive correlations
connectivity = {}
def connectivity_matrix(epochs, i):
    return spectral_connectivity_epochs(
    epochs, method='pli', mode='multitaper', sfreq=SFREQ,
    fmin=FREQ_BANDS[mapping[i]][0], fmax=FREQ_BANDS[mapping[i]][1], faverage=True, n_jobs=1)


In [12]:
n_channels = len(raw_1.info['ch_names'])
for i in mapping:
    con = connectivity_matrix(epochs_1, i)
    connectivity[f'{GROUP1}-{mapping[i]}'] = con.get_data().reshape((n_channels, n_channels))

    con = connectivity_matrix(epochs_2, i)
    connectivity[f'{GROUP2}-{mapping[i]}'] = con.get_data().reshape((n_channels, n_channels))

    # Make matrix symmetric
    for row in range(n_channels):
        for col in range(n_channels):
            connectivity[f'{GROUP1}-{mapping[i]}'][row][col] = connectivity[f'{GROUP1}-{mapping[i]}'][col][row]
            connectivity[f'{GROUP2}-{mapping[i]}'][row][col] = connectivity[f'{GROUP2}-{mapping[i]}'][col][row]


Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 0.2Hz..4.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epo

/tmp/ipykernel_16975/203938963.py:4: RuntimeWarning: fmin=0.010 Hz corresponds to 0.040 < 5 cycles based on the epoch length 4.002 sec, need at least 500.000 sec epochs or fmin=1.249. Spectrum estimate will be unreliable.
  return spectral_connectivity_epochs(


    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
    computing connectivity for epoch 32
    computing connectivity for epoch 33
    computing connectivity for epoch 34
    assembling connectivity matrix
[Connectivity computation done]
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 0.2Hz..4.0Hz (16 points)
  

/tmp/ipykernel_16975/203938963.py:4: RuntimeWarning: fmin=0.010 Hz corresponds to 0.040 < 5 cycles based on the epoch length 4.002 sec, need at least 500.000 sec epochs or fmin=1.249. Spectrum estimate will be unreliable.
  return spectral_connectivity_epochs(


    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
    computing connectivity for epoch 32
    computing connectivity for epoch 33
    computing connectivity for epoch 34
    assembling connectivity matrix
[Connectivity computation done]
Replacing existing metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS wi

### 3. Thresholding - M1 -  Small World Index

Preserves small-world properties in both groups to ensure that any observed differences in connectivity are not biased by the thresholding method.
Preserving these properties ensures that information can be transmitted quickly and effectively across the network.

In [13]:
optimal_threshold = {}
edges_1 = {}
edges_2 = {}
# Generating 50 random networks with same number of vertices and edges 
random_networks = [nx.gnm_random_graph(n_channels, 496) for _ in range(50)]
thresholds = np.arange(0.1, 0.9, 0.002)

for i in mapping:
    edges_1[mapping[i]] = []
    edges_2[mapping[i]] = []
    ratios = []
    for threshold in thresholds:
        # TODO: Cross check should we ignore thresholds which result in matrix to be not connected ?
        g1 = calculate_avergae_components(nx.from_numpy_array(connectivity[f'{GROUP1}-{mapping[i]}'] > threshold))
        Lw_1_binarized, CC_1_binarized = g1[0], g1[3]

        g2 = calculate_avergae_components(nx.from_numpy_array(connectivity[f'{GROUP2}-{mapping[i]}'] > threshold))
        Lw_2_binarized, CC_2_binarized = g2[0], g2[3]

        swi_ratio_binarized =  (CC_1_binarized*Lw_2_binarized)/(Lw_1_binarized*CC_2_binarized) if Lw_1_binarized and CC_2_binarized else 0
        ratios.append(swi_ratio_binarized)
        
    optimal_threshold[mapping[i]] = thresholds[np.argmax(np.abs(ratios))] #TODO: ratios - swi_ratio ? OR remove swi_ratio and related code above
    t_stat, p_value = ttest_ind(np.mean(connectivity[f'{GROUP1}-{mapping[i]}'], axis=0), np.mean(connectivity[f'{GROUP2}-{mapping[i]}'], axis=0)) 
    if p_value < 0.005: # Reject the null hypotheses
        Crand = np.mean([calculate_avergae_components(nx.from_numpy_array((nx.to_numpy_array(G) > optimal_threshold[mapping[i]])))[3] for G in random_networks])
        Lrand = np.mean([calculate_avergae_components(nx.from_numpy_array((nx.to_numpy_array(G) > optimal_threshold[mapping[i]])))[0] for G in random_networks])

        graph1 = calculate_avergae_components(nx.from_numpy_array(connectivity[f'{GROUP1}-{mapping[i]}'] > optimal_threshold[mapping[i]]))
        Lw_1_binarized, CC_1_binarized = graph1[0], graph1[3]

        graph2 = calculate_avergae_components(nx.from_numpy_array(connectivity[f'{GROUP2}-{mapping[i]}'] > optimal_threshold[mapping[i]]))
        Lw_2_binarized, CC_2_binarized = graph2[0], graph2[3]

        swi_binarized_1 =  (CC_1_binarized/Crand)/(Lw_1_binarized/Lrand)
        swi_binarized_2 = (CC_2_binarized/Crand)/(Lw_2_binarized/Lrand)
        print(swi_binarized_1, "Cc1 ", CC_1_binarized, "Lw1 ", Lw_1_binarized)
        print(swi_binarized_2, "Cc2 ", CC_2_binarized, "Lw2 ", Lw_2_binarized)  

        edges_1[mapping[i]].append(nx.from_numpy_array(connectivity[f'{GROUP1}-{mapping[i]}'] > optimal_threshold[mapping[i]]).number_of_edges())
        edges_2[mapping[i]].append(nx.from_numpy_array(connectivity[f'{GROUP2}-{mapping[i]}'] > optimal_threshold[mapping[i]]).number_of_edges())

        print(f"{mapping[i]} - Optimal Threshold: {optimal_threshold[mapping[i]]} (Significant)", p_value)
        row_data[('swi', f'{mapping[i]}-g1')] = swi_binarized_1
        row_data[('swi', f'{mapping[i]}-g2')] = swi_binarized_2
    else:
        row_data[('swi', f'{mapping[i]}-g1')] = None
        row_data[('swi', f'{mapping[i]}-g2')] = None

0.33432626598913606 Cc1  0.07708468002585649 Lw1  0.23056722689075632
0.065625 Cc2  0.014035087719298246 Lw2  0.21386800334168754
delta - Optimal Threshold: 0.3620000000000002 (Significant) 0.0007423443023424042
0.27540464742468623 Cc1  0.5052786878154526 Lw1  1.8346774193548387
0.0370564281559046 Cc2  0.008356290174471993 Lw2  0.2255017709563164
alpha1 - Optimal Threshold: 0.5240000000000004 (Significant) 1.9324140779789777e-09
0.29610995855523947 Cc1  0.5056555139038061 Lw1  1.7076612903225807
0.04974682334957486 Cc2  0.03562534209085934 Lw2  0.7161330049261084
alpha2 - Optimal Threshold: 0.4320000000000003 (Significant) 5.469661407857707e-11
0.17076118326118328 Cc1  0.04098268398268399 Lw1  0.24
0.033928571428571426 Cc2  0.008412042502951593 Lw2  0.24793388429752064
beta1 - Optimal Threshold: 0.6120000000000004 (Significant) 6.104746051563774e-05
0.1963751381115398 Cc1  0.06685706269039603 Lw1  0.34045584045584043
0.01398228030110801 Cc2  0.007130456349206349 Lw2  0.509963768115942


### 4. Binarization of functional connectivity

In [14]:
binarized_matrix = {}
for i in mapping:
    binarized_matrix[f'{GROUP1}-{mapping[i]}'] = connectivity[f'{GROUP1}-{mapping[i]}'] > optimal_threshold[mapping[i]]
    binarized_matrix[f'{GROUP2}-{mapping[i]}'] = connectivity[f'{GROUP2}-{mapping[i]}'] > optimal_threshold[mapping[i]]
print(binarized_matrix.keys())

dict_keys(['2-delta', '3-delta', '2-theta', '3-theta', '2-alpha1', '3-alpha1', '2-alpha2', '3-alpha2', '2-beta1', '3-beta1', '2-beta2', '3-beta2', '2-beta3', '3-beta3', '2-gamma', '3-gamma'])


Find number of links in each graph

In [15]:
for i in mapping:
    links_1 = 0
    links_2 = 0

    for row in range(n_channels):
        for col in range(row+1):
            if binarized_matrix[f'{GROUP1}-{mapping[i]}'][row][col] == 1:
                links_1+=1
            if binarized_matrix[f'{GROUP2}-{mapping[i]}'][row][col] == 1:
                links_2+=1

    print(f'{GROUP1}-{mapping[i]}-number of links: K=', links_1)
    print(f'{GROUP2}-{mapping[i]}-number of links: K=', links_2)

    row_data[('links', f'{mapping[i]}-g1')] = links_1
    row_data[('links', f'{mapping[i]}-g2')] = links_2

2-delta-number of links: K= 21
3-delta-number of links: K= 27
2-theta-number of links: K= 68
3-theta-number of links: K= 24
2-alpha1-number of links: K= 143
3-alpha1-number of links: K= 25
2-alpha2-number of links: K= 170
3-alpha2-number of links: K= 40
2-beta1-number of links: K= 73
3-beta1-number of links: K= 23
2-beta2-number of links: K= 76
3-beta2-number of links: K= 25
2-beta3-number of links: K= 99
3-beta3-number of links: K= 72
2-gamma-number of links: K= 41
3-gamma-number of links: K= 20


### 5. Asymmetry in  each band

In [16]:
left_region = ['Fp1', 'AF3', 'PO3','O1', 'CP1', 'FC1', 'P3', 'C3', 'F3', 'F7', 'FC5', 'CP5', 'T7', 'P7']
right_region = ['P8', 'T8', 'CP6', 'FC6', 'F8', 'F4', 'C4', 'P4', 'AF4', 'Fp2', 'FC2', 'CP2', 'O2', 'PO4']

for i in mapping:
    lr = 1
    rr = 1
    lrc = 1
    rrc = 1
    for row in range(n_channels):
        for col in range(row+1):
            if binarized_matrix[f'{GROUP1}-{mapping[i]}'][row][col] == 1:
                if raw_1.info['ch_names'][row] in left_region or raw_1.info['ch_names'][col] in left_region:
                    lr += 1
                if raw_1.info['ch_names'][row] in right_region or raw_1.info['ch_names'][col] in right_region:
                    rr += 1
                # print(raw_1.info['ch_names'][row], raw_1.info['ch_names'][col])

            if binarized_matrix[f'{GROUP2}-{mapping[i]}'][row][col] == 1:
                if raw_1.info['ch_names'][row] in left_region or raw_1.info['ch_names'][col] in left_region:
                    lrc += 1
                if raw_1.info['ch_names'][row] in right_region or raw_1.info['ch_names'][col] in right_region:
                    rrc += 1
                # print(raw_1.info['ch_names'][row], raw_1.info['ch_names'][col])

    print(f"{GROUP1}: ",mapping[i], "left region: ", lr, "right region: ", rr, "proportion: ", lr/rr)
    print(f"{GROUP2}: ", mapping[i], "left region: ", lrc, "right region: ", rrc, "proportion: ", lrc/rrc)

    row_data[('asymmetry', f'{mapping[i]}-g1')] = lr/rr
    row_data[('asymmetry', f'{mapping[i]}-g2')] = lrc/rrc

2:  delta left region:  13 right region:  19 proportion:  0.6842105263157895
3:  delta left region:  18 right region:  24 proportion:  0.75
2:  theta left region:  39 right region:  54 proportion:  0.7222222222222222
3:  theta left region:  20 right region:  12 proportion:  1.6666666666666667
2:  alpha1 left region:  106 right region:  89 proportion:  1.1910112359550562
3:  alpha1 left region:  18 right region:  12 proportion:  1.5
2:  alpha2 left region:  116 right region:  115 proportion:  1.008695652173913
3:  alpha2 left region:  28 right region:  23 proportion:  1.2173913043478262
2:  beta1 left region:  49 right region:  51 proportion:  0.9607843137254902
3:  beta1 left region:  10 right region:  17 proportion:  0.5882352941176471
2:  beta2 left region:  53 right region:  47 proportion:  1.127659574468085
3:  beta2 left region:  12 right region:  16 proportion:  0.75
2:  beta3 left region:  65 right region:  56 proportion:  1.1607142857142858
3:  beta3 left region:  47 right regi

### 6. Difference matrix

In [17]:
def return_index(channels_list, raw):
    indexes = []
    for channel in channels_list:
        for i in range(len(raw.info['ch_names'])):
            if raw.info['ch_names'][i]==channel:
                indexes.append(i)
    return(indexes)

In [18]:
central_channels = return_index(['Pz', 'Oz', 'Fz', 'Cz'], raw_1)
central_channels
raw_1.info['bads'] = []

In [19]:
difference_matrix = {}
for i in mapping:
    difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) 
    # difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int)
    for k in range(n_channels):
        for l in range(n_channels):
            if difference_matrix[mapping[i]][k][l] == -1 or k in central_channels or l in central_channels:
                difference_matrix[mapping[i]][k][l] = 0

    # plot_sensors_connectivity(
    #     raw_1.info,
    #     difference_matrix[mapping[i]],
    #     cbar_label=f'{mapping[i]}-Connectivity',
    #     )

In [20]:
selected_channels = {}
for i in mapping:
    selected_channels[mapping[i]] = []
    for ch in range(32):
        # TODO: instead of number of edges wise, we can explore other algorithm
        if len(nx.from_numpy_array(difference_matrix[mapping[i]]).edges(ch)) > 5:
            selected_channels[mapping[i]].append(raw_1.info['ch_names'][ch])
print(selected_channels)

{'delta': ['P8'], 'theta': ['C3'], 'alpha1': ['C3'], 'alpha2': ['C3'], 'beta1': ['P8'], 'beta2': [], 'beta3': ['P8', 'C4', 'C3'], 'gamma': ['P8']}


Create dictionary of Regions in brain vs channels

In [21]:
regionvschannel = {
    'LC': ['C3', 'CP1', 'CP5', 'FC1', 'FC5'], # Left central 
    'LF': ['FP1', 'AF3', 'F3', 'F7'], # Left frontal
    'LT': ['T7'], # Left temporal
    'LPO': ['PO3',  'P3', 'P7', 'O1'], # Left parietal-occipital
    'RC': ['CP6', 'FC6', 'C4', 'FC2', 'CP2',], # Right central 
    'RF': ['F8', 'F4', 'AF4', 'FP2'], # Right frontal
    'RT': ['T8'], # Right temporal
    'RPO': ['P8', 'P4', 'PO4', 'O2'] , # Right parietal-occipital
}

In [22]:
channel_to_region = {}
for region, region_channels in regionvschannel.items():
    for channel in region_channels:
        channel_to_region[channel] = region

selected_regions = {}
for band, channels in selected_channels.items():
    regions = set()
    for channel in channels:
        if channel in channel_to_region:
            regions.add(channel_to_region[channel])
    selected_regions[band] = list(regions)

In [23]:
# row_data = {('index', 'index'): 'Active-Preeti-0vs1',
#  ('swi', 'delta-g1'): 0.0984126984126984,
#  ('swi', 'delta-g2'): 0.0462557603686636,
#  ('swi', 'theta-g1'): 0.04757092643735877,
#  ('swi', 'theta-g2'): 0.012385844748858447,
#  ('swi', 'alpha1-g1'): None,
#  ('swi', 'alpha1-g2'): None,
#  ('swi', 'alpha2-g1'): None,
#  ('swi', 'alpha2-g2'): None,
#  ('swi', 'beta1-g1'): None,
#  ('swi', 'beta1-g2'): None,
#  ('swi', 'beta2-g1'): None,
#  ('swi', 'beta2-g2'): None,
#  ('swi', 'beta3-g1'): None,
#  ('swi', 'beta3-g2'): None,
#  ('swi', 'gamma-g1'): None,
#  ('swi', 'gamma-g2'): None,
#  ('links', 'delta-g1'): 9,
#  ('links', 'delta-g2'): 43,
#  ('links', 'theta-g1'): 21,
#  ('links', 'theta-g2'): 35,
#  ('links', 'alpha1-g1'): 42,
#  ('links', 'alpha1-g2'): 31,
#  ('links', 'alpha2-g1'): 119,
#  ('links', 'alpha2-g2'): 61,
#  ('links', 'beta1-g1'): 126,
#  ('links', 'beta1-g2'): 174,
#  ('links', 'beta2-g1'): 101,
#  ('links', 'beta2-g2'): 88,
#  ('links', 'beta3-g1'): 57,
#  ('links', 'beta3-g2'): 63,
#  ('links', 'gamma-g1'): 30,
#  ('links', 'gamma-g2'): 37,
#  ('asymmetry', 'delta-g1'): 1.6,
#  ('asymmetry', 'delta-g2'): 1.0357142857142858,
#  ('asymmetry', 'theta-g1'): 1.0,
#  ('asymmetry', 'theta-g2'): 0.8571428571428571,
#  ('asymmetry', 'alpha1-g1'): 1.3181818181818181,
#  ('asymmetry', 'alpha1-g2'): 1.0454545454545454,
#  ('asymmetry', 'alpha2-g1'): 1.0266666666666666,
#  ('asymmetry', 'alpha2-g2'): 0.8837209302325582,
#  ('asymmetry', 'beta1-g1'): 0.9156626506024096,
#  ('asymmetry', 'beta1-g2'): 1.0625,
#  ('asymmetry', 'beta2-g1'): 1.1355932203389831,
#  ('asymmetry', 'beta2-g2'): 1.4130434782608696,
#  ('asymmetry', 'beta3-g1'): 1.5185185185185186,
#  ('asymmetry', 'beta3-g2'): 1.2352941176470589,
#  ('asymmetry', 'gamma-g1'): 1.1666666666666667,
#  ('asymmetry', 'gamma-g2'): 1.0,
#  ('regions', 'delta'): ['LC', 'RC'],
#  ('regions', 'theta'): ['LC'],
#  ('regions', 'alpha1'): [],
#  ('regions', 'alpha2'): [],
#  ('regions', 'beta1'): ['RT', 'LC', 'RF', 'LF', 'RC', 'LPO'],
#  ('regions', 'beta2'): [],
#  ('regions', 'beta3'): [],
#  ('regions', 'gamma'): []}

In [24]:
for i in mapping:
    row_data[('regions', mapping[i])] = selected_regions[mapping[i]]
row_data

{('index', 'index'): 'Active-Preeti-2vs3',
 ('swi', 'delta-g1'): 0.33432626598913606,
 ('swi', 'delta-g2'): 0.065625,
 ('swi', 'theta-g1'): None,
 ('swi', 'theta-g2'): None,
 ('swi', 'alpha1-g1'): 0.27540464742468623,
 ('swi', 'alpha1-g2'): 0.0370564281559046,
 ('swi', 'alpha2-g1'): 0.29610995855523947,
 ('swi', 'alpha2-g2'): 0.04974682334957486,
 ('swi', 'beta1-g1'): 0.17076118326118328,
 ('swi', 'beta1-g2'): 0.033928571428571426,
 ('swi', 'beta2-g1'): 0.1963751381115398,
 ('swi', 'beta2-g2'): 0.01398228030110801,
 ('swi', 'beta3-g1'): None,
 ('swi', 'beta3-g2'): None,
 ('swi', 'gamma-g1'): None,
 ('swi', 'gamma-g2'): None,
 ('links', 'delta-g1'): 21,
 ('links', 'delta-g2'): 27,
 ('links', 'theta-g1'): 68,
 ('links', 'theta-g2'): 24,
 ('links', 'alpha1-g1'): 143,
 ('links', 'alpha1-g2'): 25,
 ('links', 'alpha2-g1'): 170,
 ('links', 'alpha2-g2'): 40,
 ('links', 'beta1-g1'): 73,
 ('links', 'beta1-g2'): 23,
 ('links', 'beta2-g1'): 76,
 ('links', 'beta2-g2'): 25,
 ('links', 'beta3-g1'): 9

In [ ]:
(df)

In [ ]:
# df = df.drop(df.index)
new_row = pd.Series(row_data)
df.loc[len(df)] = new_row

In [ ]:
df.to_excel(existing_file, index=True, header=True)
df